In [ ]:
# pip install load_dotenv
# pip install google-api-python-client
# pip install google-auth-oauthlib
# pip install streamlit

In [ ]:
import json
import requests
import os
import re
import json
import pickle
from dotenv import load_dotenv
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from datetime import datetime, timedelta

def get_response(prompt):
    """
    To get response from the model via API call from Ollama and generate response from LLM with the given prompt.
    
    Parameters:
    prompt (str): The prompt to generate response.
    
    Returns:
    str: The generated response.
    """
    
    url = "http://127.0.0.1:11434/v1/completions"
    headers = {
        "Content-Type": "application/json"
        }
    data = {
        "prompt": prompt,
        "model": "iodose/nuextract-v1.5"
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        response_data = response.json()
        return response_data["choices"][0]["text"]
    else:
        return f"Error: {response.status_code}, {response.text}"

In [51]:
def extract_appointment_details(message):
    json_format = """
    {
    "appointment":
    "location": 
    "date": DD MMM YYYY (example: 5 FEB 2024)
    "time": 
    "description": 
    }
    """
    prompt = f"""Extract the following text and return in a JSON format, no need for new line
    {message}

    Example JSON format:
    {json_format}
    """
    while True:
        try:
            response = get_response(prompt)
            appointment_details = json.loads(response)
            
            # Check if the date is in "DD MMM YYYY" format
            date_str = appointment_details.get("date", "")
            if not re.fullmatch(r"\d{1,2} [A-Za-z]{3} \d{4}", date_str):
                print("Date format does not match expected 'DD MMM YYYY' format, retrying...")
                continue

            # Check for any error in the JSON response
            if "Error" in appointment_details:
                print("Error detected, retrying...")
                continue
            else:
                break
        except json.JSONDecodeError:
            print("JSONDecodeError detected, retrying...")
        except Exception as e:
            print(f"An unexpected error occurred: {e}, retrying...")
    return appointment_details

In [ ]:
def json_checker(json_data):
    if json_data.get("appointment") is None:
        print("Using Default Appointment Name")
        json_data["appointment"] = "Health Appointment"
    return json_data

def get_google_calendar_service():
    """Authenticate and return a Google Calendar service instance."""
    load_dotenv()
    SCOPES = ["https://www.googleapis.com/auth/calendar"]

    flow = InstalledAppFlow.from_client_config(
        {
            "installed": {
                "client_id": os.getenv("GOOGLE_CLIENT_ID"),
                "client_secret": os.getenv("GOOGLE_CLIENT_SECRET"),
                "auth_uri": os.getenv("GOOGLE_AUTH_URI"),
                "token_uri": os.getenv("GOOGLE_TOKEN_URI"),
                "auth_provider_x509_cert_url": os.getenv("GOOGLE_AUTH_PROVIDER_CERT_URL"),
                "redirect_uris": ["http://localhost"],
            }
        },
        SCOPES,
    )

    if os.path.exists("token.pickle"):
        credentials = pickle.load(open("token.pickle", "rb"))
    else:
        credentials = flow.run_local_server(port=8080)
        with open("token.pickle", "wb") as token:
            pickle.dump(credentials, token)

    service = build("calendar", "v3", credentials=credentials)
    return service

def build_event_body(event_details: dict, duration: int) -> dict:
    """Construct the event body required by the Google Calendar API."""
    # Use json_checker to set defaults as needed.
    event_details = json_checker(event_details)
    
    # Combine date and time, then convert to datetime objects.
    start_time_str = f"{event_details['date']} {event_details['time']}"
    start_time = datetime.strptime(start_time_str, "%d %b %Y %I:%M %p")
    end_time = start_time + timedelta(hours=duration)
    
    event_body = {
        "summary": event_details.get("appointment"),
        "start": {
            "dateTime": start_time.isoformat(),
            "timeZone": event_details.get("timeZone", "Asia/Singapore"),
        },
        "end": {
            "dateTime": end_time.isoformat(),
            "timeZone": event_details.get("timeZone", "Asia/Singapore"),
        },
        "description": event_details.get("description") or "",
        "location": event_details.get("location") or "",
        "reminders": {
            "useDefault": False,
            "overrides": [
                {"method": "email", "minutes": 1440},
                {"method": "popup", "minutes": 10},
            ],
        },
    }
    return event_body

def create_calendar_event(event_details: dict, duration: int) -> None:
    """Creates an event on Google Calendar using provided details."""
    service = get_google_calendar_service()
    event_body = build_event_body(event_details, duration)

    try:
        created_event = service.events().insert(calendarId="primary", body=event_body).execute()
        print("\n\033[92mEvent successfully created on Google Calendar!\033[0m")
        print(f"View event online: {created_event.get('htmlLink')}\n")
    except Exception as e:
        print(f"Error creating event: {e}")
        


In [ ]:
appointment_details = extract_appointment_details(message)
create_calendar_event(appointment_details, 1)


Event successfully created on Google Calendar!
View event online: https://www.google.com/calendar/event?eid=MzAxYmtqdnAwczA5cGU5ODEyamhuOHZtczQgb25ueXVuaHVpQG0



In [ ]:


#message = "Dear ZHENG HAOFENG, you have an appt on WED/19 August 2025, 09:30 AM at Punggol Polyclinic. View/change appts on Health Buddy app at https://for.sg/dl-hb or call 6643 6969"
message = "Dear Ms. DIANE, You have a First Visit Consultation at ENT-Head & Neck Surg Ctr - 15C, NUH Medical Centre, Zone B, Level 15, 15c, Lift Lobby B2 on 19 Feb 2025 at 3:45 pm."

def main(message):
    appointment_details = extract_appointment_details(message)
    duration =1
    create_calendar_event(appointment_details, duration)

main(message)



Event successfully created on Google Calendar!
View event online: https://www.google.com/calendar/event?eid=OGY1Zm1nOGRkNGdwN2d1ZGVrb3Mwb3VuczAgb25ueXVuaHVpQG0



In [44]:
import streamlit as st

def chatbot():
    st.title("Appointment Chatbot")

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat messages from history on app rerun
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # React to user input
    if prompt := st.chat_input("What is your appointment?"):
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})
        # Display user message in chat message container
        with st.chat_message("user"):
            st.markdown(prompt)

        # Extract appointment details
        json_format = """
            {
            "appointment":
            "location": 
            "date": DD MMM YYYY (example: 5 FEB 2024)
            "time": 
            "description": 
            }
            """
        appointment_details = main(prompt, json_format)

        # Create calendar event
        duration = 2  # Example duration in hours
        create_calendar_event(appointment_details, duration)

        # Add assistant response to chat history
        response = f"Appointment details: {appointment_details}"
        st.session_state.messages.append({"role": "assistant", "content": response})
        # Display assistant response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)